In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyproj import Proj, transform
import folium
from folium.plugins import MarkerCluster

In [3]:
full_df = pd.read_csv(r"D:\DataScienceKurs\CapitalBikeShare\datasets\CBS_2021-2023_Full.csv", sep=",")

D:\Users\Olunia\AppData\Local\Temp\ipykernel_9424\3866548688.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  full_df = pd.read_csv(r"D:\DataScienceKurs\CapitalBikeShare\datasets\CBS_2021-2023_Full.csv", sep=",")


In [4]:
elevation_df = pd.read_csv("..\\src\\elevation_data.csv")

In [5]:
station_names = full_df.drop_duplicates(subset=["start_station_name"])


In [6]:
station_names = station_names[["start_station_name","start_lat","start_lng"]]
station_names

,start_station_name,start_lat,start_lng
0,Maine Ave & 9th St SW,38.880440,-77.025236
1,10th & U St NW,38.917193,-77.025894
2,17th & Corcoran St NW,38.912138,-77.038568
3,Wilson Blvd. & N. Vermont St.,38.879477,-77.114563
4,11th & Park Rd NW,38.931322,-77.028247
...,...,...,...
10561781,Old Meadow Rd & Cathy Ln,38.916908,-77.213879
10623902,Route 29 & Circle Woods Dr,38.869114,-77.271538
10628992,Prosperity Ave & Avenir Pl,38.881436,-77.230377
10629316,Kingsbridge Dr & Draper Dr,38.868545,-77.282083


In [7]:
station_names.reset_index(drop =True, inplace =True)

In [8]:
station_names

,start_station_name,start_lat,start_lng
0,Maine Ave & 9th St SW,38.880440,-77.025236
1,10th & U St NW,38.917193,-77.025894
2,17th & Corcoran St NW,38.912138,-77.038568
3,Wilson Blvd. & N. Vermont St.,38.879477,-77.114563
4,11th & Park Rd NW,38.931322,-77.028247
...,...,...,...
856,Old Meadow Rd & Cathy Ln,38.916908,-77.213879
857,Route 29 & Circle Woods Dr,38.869114,-77.271538
858,Prosperity Ave & Avenir Pl,38.881436,-77.230377
859,Kingsbridge Dr & Draper Dr,38.868545,-77.282083


In [9]:
# import requests
# import pandas as pd
# import time
# from dotenv import load_dotenv
# import os

# # Load environment variables for your API key
# load_dotenv()

# # Retrieve the API key securely from environment variables
# api_key = os.getenv('OPENROUTESERVICE_API_KEY')

# # Check if the API key is found
# if not api_key:
#     print("API key not found.")
#     exit()

# # Define headers for the request
# headers = {
#     'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
# }

# # Assuming your station_names DataFrame has these columns: start_lng, start_lat, start_station_name
# # Prepare to store results
# elevation_data = []

# # Counter to track the number of requests made
# request_count = 0

# # Loop through your DataFrame to get the coordinates and make the API request
# for index, row in station_names.iterrows():
#     # Get the coordinates from your DataFrame
#     coord = f"{row['start_lng']},{row['start_lat']}"
#     print(f"Fetching elevation for station: {row['start_station_name']} at coordinates {coord}")

#     # Construct the API URL for the elevation endpoint
#     url = f'https://api.openrouteservice.org/elevation/point?api_key={api_key}&geometry={coord}&format_out=point&dataset=srtm'
    
#     # Make the API request
#     response = requests.get(url, headers=headers)
    
#     if response.status_code == 200:
#         # Parse the response data
#         data = response.json()
        
#         # Check if the response contains the elevation data
#         if 'geometry' in data and len(data['geometry']) > 2:
#             elevation = data['geometry'][2]  # Elevation is the third value in the geometry list
#             elevation_data.append({
#                 'start_station_name': row['start_station_name'],  # Use station name from the DataFrame
#                 'latitude': row['start_lat'],
#                 'longitude': row['start_lng'],
#                 'elevation': elevation
#             })
#             print(f"Elevation for {row['start_station_name']} at {coord}: {elevation} meters")
#         else:
#             print(f"Unexpected response structure for {coord}. Skipping this coordinate.")
#     else:
#         print(f"Error for coordinate {coord}: {response.status_code} - {response.text}")
    
#     # Increase request count
#     request_count += 1
    
#     # Check if we've made 100 requests, then sleep for a minute
#     if request_count >= 100:
#         print("Reached 100 requests. Sleeping for 60 seconds to respect rate limit...")
#         time.sleep(60)  # Sleep for 60 seconds
#         request_count = 0  # Reset the request count

#     # To avoid exceeding the API rate limit, sleep for a short period between requests (e.g., 0.6 seconds)
#     time.sleep(0.6)

# # Convert the result into a DataFrame
# elevation_df = pd.DataFrame(elevation_data)

# # Show the results
# print(elevation_df)



In [10]:
elevation_df.head()

,station_name,latitude,longitude,elevation,elevation_label
0,Maine Ave & 9th St SW,38.880440,-77.025236,3.0,low
1,10th & U St NW,38.917193,-77.025894,31.0,medium
2,17th & Corcoran St NW,38.912138,-77.038568,29.0,medium
3,Wilson Blvd. & N. Vermont St.,38.879477,-77.114563,82.0,high
4,11th & Park Rd NW,38.931322,-77.028247,60.0,medium


In [11]:
# Check if there are any NaN values in the 'elevation' column
print(elevation_df['elevation'].isna().sum())  # Correct method to check for NaN values


0


In [12]:
# Wysokości, które podzielą dane na 3 grupy
low_elevation = elevation_df[(elevation_df['elevation'] < 19.43)]
medium_elevation = elevation_df[(elevation_df['elevation'] >= 19.43) & (elevation_df['elevation'] <= 70.49)]
high_elevation = elevation_df[elevation_df['elevation'] > 70.49]

# Możesz teraz stworzyć różne analizy dla tych trzech grup
print("Low elevation stats:")
print(low_elevation['elevation'].describe())

print("\nMedium elevation stats:")
print(medium_elevation['elevation'].describe())

print("\nHigh elevation stats:")
print(high_elevation['elevation'].describe())


Low elevation stats:
count    203.000000
mean      11.615764
std        5.063962
min       -3.000000
25%        8.000000
50%       12.000000
75%       16.000000
max       19.000000
Name: elevation, dtype: float64

Medium elevation stats:
count    346.000000
mean      40.341040
std       14.781869
min       20.000000
25%       28.000000
50%       36.000000
75%       54.000000
max       70.000000
Name: elevation, dtype: float64

High elevation stats:
count    310.000000
mean     107.470968
std       21.547963
min       71.000000
25%       89.000000
50%      109.000000
75%      122.000000
max      161.000000
Name: elevation, dtype: float64


In [13]:
# Dodajemy nową kolumnę 'elevation_label', która będzie zawierać etykiety
def classify_elevation(elevation):
    if elevation < 19.43:
        return 'low'
    elif elevation <= 70.49:
        return 'medium'
    else:
        return 'high'

# Zastosowanie funkcji do kolumny 'elevation'
elevation_df['elevation_label'] = elevation_df['elevation'].apply(classify_elevation)

# Sprawdzenie wyników
print(elevation_df[['latitude', 'longitude', 'elevation', 'elevation_label']].head())


    latitude  longitude  elevation elevation_label
0  38.880440 -77.025236        3.0             low
1  38.917193 -77.025894       31.0          medium
2  38.912138 -77.038568       29.0          medium
3  38.879477 -77.114563       82.0            high
4  38.931322 -77.028247       60.0          medium


In [14]:
elevation_df.head()

,station_name,latitude,longitude,elevation,elevation_label
0,Maine Ave & 9th St SW,38.880440,-77.025236,3.0,low
1,10th & U St NW,38.917193,-77.025894,31.0,medium
2,17th & Corcoran St NW,38.912138,-77.038568,29.0,medium
3,Wilson Blvd. & N. Vermont St.,38.879477,-77.114563,82.0,high
4,11th & Park Rd NW,38.931322,-77.028247,60.0,medium


In [15]:
elevation_df.rename(columns={"start_station_name": "station_name"}, inplace = True)

In [16]:
elevation_df

,station_name,latitude,longitude,elevation,elevation_label
0,Maine Ave & 9th St SW,38.880440,-77.025236,3.0,low
1,10th & U St NW,38.917193,-77.025894,31.0,medium
2,17th & Corcoran St NW,38.912138,-77.038568,29.0,medium
3,Wilson Blvd. & N. Vermont St.,38.879477,-77.114563,82.0,high
4,11th & Park Rd NW,38.931322,-77.028247,60.0,medium
...,...,...,...,...,...
854,Old Meadow Rd & Cathy Ln,38.916908,-77.213879,121.0,high
855,Route 29 & Circle Woods Dr,38.869114,-77.271538,102.0,high
856,Prosperity Ave & Avenir Pl,38.881436,-77.230377,119.0,high
857,Kingsbridge Dr & Draper Dr,38.868545,-77.282083,98.0,high


In [19]:
elevation_df[(elevation_df["latitude"] == "38.960000") & elevation_df["longitude"] =="-77.020000"]

,station_name,latitude,longitude,elevation,elevation_label


In [18]:
elevation_df.to_csv('elevation_data.csv', index = False)